In [1]:
# geospatial analysis
import geopandas as gpd
import shapely

# working with files
import os

# data analysis
import pandas as pd

# web scraping
import requests
from bs4 import BeautifulSoup
import time
import re

### Essential Variables

In [2]:
# let's try to convert everything to EPSG:5070. For more accurate area measurements

# boundary for state of California
us_states = gpd.read_file("Data/Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
us_states.to_crs("EPSG:5070", inplace=True)
ca_state = us_states[us_states["STUSPS"] == "CA"]

# California counties
us_counties = gpd.read_file("Data/Boundaries/cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
ca_counties = us_counties[us_counties["STATEFP"] == "06"]

# HUC8 subbasins
huc8_ca = gpd.read_file("Data/Boundaries/ca_HUC8.geojson")

In [3]:
# input data (should we make this a JSON or yaml file later?). Note we ONLY look at operational costs

data_center_cost = 12e6 # ($/MW of capacity)

### Read in Data

Emissions and Water Scarcity

In [6]:
# # let's go with Siddik et al. data (2021) for now.
# water_carbon = pd.read_excel("Data/Footprint/Siddik/SI_XLS/Results.xlsx", sheet_name="Table 3", skiprows=1)

# # filter to only HUC rows
# water_carbon = water_carbon[~water_carbon['WSF_1MW_DC'].isna()]

# # turn HUC8 into string format
# water_carbon['HUC8_str'] = water_carbon['HUC8'].map(lambda x: ''.join(['0']*(8-len(str(x)))) + str(x))

# # merge with California data
# huc8_ca = huc8_ca.merge(water_carbon, how="inner", left_on="HUC8", right_on="HUC8_str")

# read in the derived values
footprint_df = pd.read_csv("Data/Footprint/footprint_per_MW.csv")

Prices

In [ ]:
# read in price data

### Land Restrictions

In summary, we can basically ignore land property restrictions.

We haven't investigated hosting capacity yet.

In [ ]:
# let's not worry about utilities for now, just for time. and I don't think it's the most important restriction
# # utilities map in California
# ca_utilities = gpd.read_file("Data/Boundaries/ElectricLoadServingEntities_IOU_POU_-6992589901327565970.geojson")

In [ ]:
# federal, state, local lands
# not too many tribal lands in California so we don't worry about that

if False:
    # overall takes 30 seconds
    us_lands = gpd.read_file("Data/Boundaries/USA_Federal_Lands_1879006603790792540.geojson")

    # filter invalid geometries
    us_lands = us_lands[~us_lands['geometry'].isna()]
    us_lands = us_lands[us_lands['geometry'].is_valid]

    us_lands.to_crs("EPSG:5070", inplace=True)

    # first filter US lands to California
    ca_geometry = ca_state.iloc[0].geometry
    us_lands_ca = us_lands[us_lands.intersects(ca_geometry)]

In [ ]:
# cut out US lands from California subbasins

# overall, turns out that no HUC8 (besides some island) is fully contained in federal land.

if False:
    # takes around 30 seconds
    all_us_land = us_lands_ca.union_all()

    subbasin_us_land = []

    for idx, row in huc8_ca.iterrows():
        if shapely.contains(all_us_land, row['geometry']):
            print(idx)
            subbasin_us_land.append(True)
        else:
            subbasin_us_land.append(False)

    huc8_ca["Government_Land"] = subbasin_us_land
    huc8_ca = huc8_ca[~huc8_ca["Government_Land"]]

355
